In [13]:
!pip install opencv-python

     ---------------------------------------- 39.5/39.5 MB 5.2 MB/s eta 0:00:00


You should consider upgrading via the 'F:\Capstone Project\capstone\Scripts\python.exe -m pip install --upgrade pip' command.


In [28]:
import json
import os
import cv2
from pathlib import Path

BASE_DIR = Path.cwd().resolve().parents[0]
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # Capstone project root
DATASET_DIR = os.path.join(BASE_DIR, "Dataset/LABELS")  # Dataset folder

def loadJSON(file_path,labelName='Labels-v2.json'):
    label_directory = Path(file_path)

    for folder in label_directory.iterdir():
        if folder.is_dir():
            league_directory = folder
            for subfolders in league_directory.iterdir():
                if subfolders.is_dir():
                    year_directory = subfolders
                    for matches in year_directory.iterdir():
                        if matches.is_dir():
                            match_directory = matches
                            JSON_path = match_directory / labelName

                            if JSON_path.exists():
                                with open(JSON_path, 'r') as f:
                                    JSON_data = json.load(f)

                                VideoURL:str = JSON_data['UrlLocal']
                                annotations = JSON_data['annotations']
                                # BASED ON VIDEO URL, find video and then trim based on annotations
                                VIDEO_DIR = os.path.join(BASE_DIR, "Dataset","VIDEOS")
                                # match_video_path = VIDEO_DIR + r"england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley"
                                match_video_path = os.path.join(VIDEO_DIR, r"england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley")
                                match_video = Path(match_video_path)
                                if match_video.exists():
                                    video_file = match_video / "1_720p.mkv"  # Construct video file path
                                    if video_file.exists():
                                        cap = cv2.VideoCapture(str(video_file))  # Convert Path to string for OpenCV

                                        if not cap.isOpened():
                                            print(f"Error: Cannot open video file at {video_file}")
                                        else:
                                            print(f"Video file opened successfully: {video_file}")

                                        cap.release()  # Release after processing
                                    else:
                                        print(f"Error: Video file not found at {video_file}")
                                else:
                                    print(f"Error: Match folder not found at {match_video}")
    print(f"URL LOCAL:{VideoURL}\n\n annotations:{annotations}")


In [30]:
loadJSON(DATASET_DIR)
# F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley

Video file opened successfully: F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv
Video file opened successfully: F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv
Video file opened successfully: F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv
Video file opened successfully: F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv
Video file opened successfully: F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv
Video file opened successfully: F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv
Video file opened successfully: F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv
Video file op

In [42]:
import json
import os
import glob
from pathlib import Path
from moviepy.video.io.VideoFileClip import VideoFileClip

# Paths
BASE_DIR = Path.cwd().resolve().parents[0]
LABELS_DIR = "path/to/LABELS"  # Folder containing JSON files
VIDEOS_DIR = os.path.join(BASE_DIR, "Dataset","VIDEOS")  # Folder containing match videos
OUTPUT_DIR = os.path.join(BASE_DIR,"Dataset","Event_clips")  # Folder where event clips will be stored

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Function to extract and save clips using MoviePy
def extract_clip(video_file, start_time, duration, output_file):
    try:
        with VideoFileClip(video_file) as video:
            start_time = min(max(0, start_time), video.duration - duration)
            end_time = min(start_time + duration, video.duration)

            clip = video.subclipped(start_time, end_time)
            clip.write_videofile(output_file, codec="libx264", audio_codec="aac", fps=30)

        print(f"✅ Extracted: {output_file}")
    except Exception as e:
        print(f"❌ Error extracting {output_file}: {e}")

# Loop through JSON files
# for json_file in glob.glob(f"{LABELS_DIR}/**/Labels-v2.json", recursive=True):
with open(r"F:\Capstone Project\Dataset\LABELS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\Labels-v2.json", "r") as f:
    labels = json.load(f)

# Extract match folder name from UrlLocal
match_folder = labels["UrlLocal"]
first_half_video = os.path.join(VIDEOS_DIR, match_folder, "1_720p.mkv")
second_half_video = os.path.join(VIDEOS_DIR, match_folder, "2_720p.mkv")

if not os.path.exists(first_half_video) or not os.path.exists(second_half_video):
    print(f"❌ Video files not found for {match_folder}")
    # continue

print(f"🎥 Processing Match: {match_folder}")

# Loop through each event annotation
for idx, annotation in enumerate(labels["annotations"]):
    gameTime = annotation["gameTime"]
    event_label = annotation["label"].replace(" ", "_")  # Remove spaces in event names
    team = annotation["team"].replace(" ", "_")

    half = int(gameTime.split(" - ")[0].strip())  # 1st or 2nd half
    time_str = gameTime.split(" - ")[1].strip()  # Extract "MM:SS"
    minutes, seconds = map(int, time_str.split(":"))
    timestamp = (minutes * 60) + seconds  # Convert to total seconds

    # Select correct video file
    video_file = first_half_video if half == 1 else second_half_video

    # Clip range (1 second before and after)
    start_time = max(0, timestamp - 4)
    duration = 8  # Total clip length (2 seconds)

    # Ensure event folder exists
    event_folder = os.path.join(OUTPUT_DIR, event_label)
    os.makedirs(event_folder, exist_ok=True)

    # Output clip name
    output_clip = os.path.join(event_folder, f"{event_label}_{idx}.mp4")

    # Extract Clip
    extract_clip(video_file, start_time, duration, output_clip)

print("🎉 All video clips extracted successfully!")


🎥 Processing Match: england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/
{'video_found': True, 'audio_found': True, 'metadata': {'ENCODER': 'Lavf56.25.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': None, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': None, 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libvorbis'}}], 'input_number': 0}], 'duration': 2700.0, 'bitrate': 2902, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': None, 'video_fps': 25.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 1, 'audio_fps': 4800

MoviePy - Done.
MoviePy - Writing video F:\Capstone Project\Dataset\Event_clips\Kick-off\Kick-off_0.mp4



MoviePy - Done !
MoviePy - video ready F:\Capstone Project\Dataset\Event_clips\Kick-off\Kick-off_0.mp4
✅ Extracted: F:\Capstone Project\Dataset\Event_clips\Kick-off\Kick-off_0.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'ENCODER': 'Lavf56.25.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': None, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': None, 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libvorbis'}}], 'input_number': 0}], 'duration': 2700.0, 'bitrate': 2902, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': None, 'video_

MoviePy - Done.
MoviePy - Writing video F:\Capstone Project\Dataset\Event_clips\Ball_out_of_play\Ball_out_of_play_1.mp4



MoviePy - Done !
MoviePy - video ready F:\Capstone Project\Dataset\Event_clips\Ball_out_of_play\Ball_out_of_play_1.mp4
✅ Extracted: F:\Capstone Project\Dataset\Event_clips\Ball_out_of_play\Ball_out_of_play_1.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'ENCODER': 'Lavf56.25.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': None, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': None, 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libvorbis'}}], 'input_number': 0}], 'duration': 2700.0, 'bitrate': 2902, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720]

...

MoviePy - Done !
MoviePy - video ready F:\Capstone Project\Dataset\Event_clips\Ball_out_of_play\Ball_out_of_play_3.mp4
✅ Extracted: F:\Capstone Project\Dataset\Event_clips\Ball_out_of_play\Ball_out_of_play_3.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'ENCODER': 'Lavf56.25.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': None, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': None, 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libvorbis'}}], 'input_number': 0}], 'duration': 2700.0, 'bitrate': 2902, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720]

MoviePy - Done.
MoviePy - Writing video F:\Capstone Project\Dataset\Event_clips\Corner\Corner_4.mp4



frame_index:  99%|█████████▉| 237/240 [00:09<00:00, 16.93it/s, now=None]

KeyboardInterrupt: 

frame_index: 100%|██████████| 240/240 [00:26<00:00, 16.93it/s, now=None]

In [34]:
import moviepy
print(moviepy.__version__)


2.1.1


In [37]:
import moviepy
from moviepy.video.io.VideoFileClip import VideoFileClip

video_file = r"F:\Capstone Project\Dataset\VIDEOS\england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_720p.mkv"
clip = VideoFileClip(video_file)  # Use an actual video file path
print(dir(clip))  # Lists all available attributes
clip.close()


{'video_found': True, 'audio_found': True, 'metadata': {'ENCODER': 'Lavf56.25.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': None, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libx264'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': None, 'metadata': {'Metadata': '', 'ENCODER': 'Lavc56.26.100 libvorbis'}}], 'input_number': 0}], 'duration': 2700.0, 'bitrate': 2902, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1280, 720], 'video_bitrate': None, 'video_fps': 25.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 1, 'audio_fps': 48000, 'audio_bitrate': None, 'video_duration': 2700.0, 'video_n_frames': 67500}
f:\Caps

In [44]:
import json
import os
import glob
from pathlib import Path
from moviepy.video.io.VideoFileClip import VideoFileClip

# Paths
BASE_DIR = Path.cwd().resolve().parents[0]
LABELS_DIR = os.path.join(BASE_DIR,"Dataset","LABELS")  # Folder containing JSON files
VIDEOS_DIR = os.path.join(BASE_DIR, "Dataset","VIDEOS")  # Folder containing match videos
OUTPUT_DIR = os.path.join(BASE_DIR,"Dataset","Event_clips")  # Folder where event clips will be stored

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Loop through JSON files
for json_file in glob.glob(f"{LABELS_DIR}/**/Labels-v2.json", recursive=True):
    with open(json_file, "r") as f:
        labels = json.load(f)

    # Extract match folder name from UrlLocal
    match_folder = labels["UrlLocal"]
    print(f"URL LOCAL:{match_folder}")

print("🎉 All video clips extracted successfully!")


URL LOCAL:england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/
URL LOCAL:england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/
URL LOCAL:england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/
URL LOCAL:england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/
URL LOCAL:england_epl/2015-2016/2015-08-08 - 19-30 Chelsea 2 - 2 Swansea/
URL LOCAL:england_epl/2015-2016/2015-08-29 - 17-00 Chelsea 1 - 2 Crystal Palace/
URL LOCAL:england_epl/2015-2016/2015-08-29 - 17-00 Manchester City 2 - 0 Watford/
URL LOCAL:england_epl/2015-2016/2015-09-12 - 14-45 Everton 3 - 1 Chelsea/
URL LOCAL:england_epl/2015-2016/2015-09-12 - 17-00 Crystal Palace 0 - 1 Manchester City/
URL LOCAL:england_epl/2015-2016/2015-09-19 - 19-30 Manchester City 1 - 2 West Ham/
URL LOCAL:england_epl/2015-2016/2015-09-26 - 17-00 Liverpool 3 - 2 Aston Villa/
URL LOCAL:england_epl/2015-2016/2015-10-17 - 17-00 Chelsea 2 - 0 Aston Villa/
URL LOCAL:england_epl/2015-2016/2015-10-